### 특정 웹툰 페이지의 모든 image를 다운로드 받기
* soup.select("img[src$='jpg']")
* img 폴더를 생성하고 그 아래에 파일을 저장하기

In [1]:
import requests
from bs4 import BeautifulSoup
import re

main_url = 'https://comic.naver.com/webtoon/detail.nhn?titleId=712362&no=94&amp;weekday=fri'
res = requests.get(main_url)
print(res.status_code)
if res.ok:
    html = res.text
    soup = BeautifulSoup(html,'html.parser')
    
    print(len(soup.select("img[src$='.jpg']")))
    img_url_list = []
    for img_url in soup.select("img[src$='.jpg']"):
        #print(type(img_url),img_url)
        # img 태그의 src 속성의 값을 가져와서 img_url_list에 추가하기
        img_url_list.append(img_url['src'])

print(len(img_url_list))        
img_url_list[0:3]   

200
301
301


['https://image-comic.pstatic.net/webtoon/712362/1/optimized_thumbnail_202x120_209eaf5e-f3bd-457b-89e3-b07a8706f8ad.jpg',
 'https://image-comic.pstatic.net/webtoon/712362/2/optimized_thumbnail_202x120_61027634-c869-405b-9455-c02e073f89f5.jpg',
 'https://image-comic.pstatic.net/webtoon/712362/3/optimized_thumbnail_202x120_f96d2800-4901-4f0a-a7af-a4342182a66e.jpg']

In [2]:
import os
#img 디렉토리 생성
#img 디렉토리가 없으면
if not os.path.isdir('img'):
    os.mkdir('img')    

In [4]:
#image_url_list에 있는 url로 요청해서 image들을 다운로드 하기
for idx,img_url in enumerate(img_url_list,1):
    #print(f'---> 다운로드 번호 {idx}')
    req_header = {
        'referer':main_url
    }
    res = requests.get(img_url, headers=req_header)
    if res.ok:
        img_data = res.content
        file_name = os.path.basename(img_url)
        with open('img/'+file_name,'wb') as file:
            #print(f'파일명 :{file_name} ({len(img_data)}) bytes')
            file.write(img_data)

### image 다운로드
* 위의 코드를 함수로 정리
* os.mkdir와 os.makedirs 
    - mkdir은 한 폴더만 생성이 가능
    - makedirs는 './a/b/c' 처럼 Sub 디렉토리를 생성이 가능

In [4]:
def download_image(title,url):
    import requests
    from bs4 import BeautifulSoup
    import os
    
    req_header = {
        'referer': url
    }
    
    res = requests.get(url)
    print(res.ok)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        imgurl_list = []
        print(len(soup.select('img[src$=".jpg"]')))
        for img_tag in soup.select('img[src$=".jpg"]'):
            imgurl_list.append(img_tag['src'])
            #print(img_tag)
                
        #image를 저장할 디렉토리 생성
        
        #dir_name = 'img/' + title
        dir_name = os.path.join('img',title)
        print(dir_name)
        if not os.path.isdir(dir_name):
            os.makedirs(dir_name)
        
        for idx,img_url in enumerate(imgurl_list,1):
            print(idx, img_url)
            res = requests.get(img_url,headers=req_header)
            #print(res.status_code)
            if res.ok:
                img_data = res.content
                #print(len(img_data))
                file_name = os.path.basename(img_url)
                #print(file_name)
                file_name = dir_name + '/' + file_name
                with open(file_name,'wb') as file:
                    print(f'{file_name} (파일크기 : {len(img_data)}))')
                    file.write(img_data)
                
                
download_image('독립일기','https://comic.naver.com/webtoon/detail?titleId=748105&no=196&weekday=thu')

True
0
img\독립일기


#### 네이버 웹툰 홈에서 추천웹툰의 제목과 url 를 알아내기

In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

main_url = 'https://comic.naver.com/index.nhn'

res = requests.get(main_url)
if res.ok:
    soup = BeautifulSoup(res.text,'html.parser')
    print(len(soup.select('div.genreRecomInfo2')))
    
    recom_webtoon_list = []
    for div_tag in soup.select('div.genreRecomInfo2'):
        recom_webtoon_dict = {}
        a_tag1 = div_tag.select('h6 a')[0]
        #웹툰제목
        title = a_tag1.text.strip()
        #웹툰의 url
        webtoon_url=urljoin(main_url, a_tag1['href'])
        
        #웹툰의 회차의 url
        a_tag2 = div_tag.select('p a')[0]
        webtoon_episode_url = urljoin(main_url,a_tag2['href'])
                
        recom_webtoon_dict['title'] = title
        recom_webtoon_dict['webtoon_url'] = webtoon_url
        recom_webtoon_dict['episode_url'] = webtoon_episode_url
        
        recom_webtoon_list.append(recom_webtoon_dict)
        
    print(recom_webtoon_list)                

0
[]


#### 6개의 만화의 제목과 회차url을 꺼내서 download_image() 함수를 호출한다.

In [5]:
# for idx,webtoon in enumerate(recom_webtoon_list,1):
#     #print('---->>> ', idx, webtoon['title'])
#     download_image(webtoon['title'],webtoon['episode_url'])

27 https://image-comic.pstatic.net/webtoon/557672/367/20210901140624_ee9e670218dc0921337e65bb07210091_IMAG01_26.jpg
img\기기괴괴/20210901140624_ee9e670218dc0921337e65bb07210091_IMAG01_26.jpg (215762))
28 https://image-comic.pstatic.net/webtoon/557672/367/20210901140624_ee9e670218dc0921337e65bb07210091_IMAG01_27.jpg
img\기기괴괴/20210901140624_ee9e670218dc0921337e65bb07210091_IMAG01_27.jpg (224546))
29 https://image-comic.pstatic.net/webtoon/557672/367/20210901140624_ee9e670218dc0921337e65bb07210091_IMAG01_28.jpg
img\기기괴괴/20210901140624_ee9e670218dc0921337e65bb07210091_IMAG01_28.jpg (63638))
30 https://image-comic.pstatic.net/banner/store/557672_PC.jpg
img\기기괴괴/557672_PC.jpg (41269))
31 https://shared-comic.pstatic.net/thumb/webtoon/557672/364/thumbnail_202x120_2d04f4b8-e6eb-42c7-a332-e7c8d6f23cac.jpg
img\기기괴괴/thumbnail_202x120_2d04f4b8-e6eb-42c7-a332-e7c8d6f23cac.jpg (20385))
32 https://shared-comic.pstatic.net/thumb/webtoon/557672/365/thumbnail_202x120_050290e4-9c7a-426e-9d46-c43f7e816c88.jpg

34 https://image-comic.pstatic.net/webtoon/733033/109/20210818104254_914860f0a50eff992101870bd749d50d_IMAG01_33.jpg
img\로그아웃/20210818104254_914860f0a50eff992101870bd749d50d_IMAG01_33.jpg (113246))
35 https://image-comic.pstatic.net/webtoon/733033/109/20210818104254_914860f0a50eff992101870bd749d50d_IMAG01_34.jpg
img\로그아웃/20210818104254_914860f0a50eff992101870bd749d50d_IMAG01_34.jpg (133752))
36 https://image-comic.pstatic.net/webtoon/733033/109/20210818104254_914860f0a50eff992101870bd749d50d_IMAG01_35.jpg
img\로그아웃/20210818104254_914860f0a50eff992101870bd749d50d_IMAG01_35.jpg (181091))
37 https://image-comic.pstatic.net/webtoon/733033/109/20210818104254_914860f0a50eff992101870bd749d50d_IMAG01_36.jpg
img\로그아웃/20210818104254_914860f0a50eff992101870bd749d50d_IMAG01_36.jpg (116157))
38 https://image-comic.pstatic.net/webtoon/733033/109/20210818104254_914860f0a50eff992101870bd749d50d_IMAG01_37.jpg
img\로그아웃/20210818104254_914860f0a50eff992101870bd749d50d_IMAG01_37.jpg (135621))
39 https://imag

35 https://shared-comic.pstatic.net/thumb/webtoon/740132/56/thumbnail_202x120_d4a43595-8391-4562-95db-ec0272090e18.jpg
img\중독연구소/thumbnail_202x120_d4a43595-8391-4562-95db-ec0272090e18.jpg (16952))
36 https://shared-comic.pstatic.net/thumb/webtoon/740132/57/thumbnail_202x120_41c1b45e-cbd1-477b-ab82-f5ab6d633e7d.jpg
img\중독연구소/thumbnail_202x120_41c1b45e-cbd1-477b-ab82-f5ab6d633e7d.jpg (12547))
37 https://shared-comic.pstatic.net/thumb/webtoon/740132/58/thumbnail_202x120_e1df83d4-b26f-4584-baba-6f9fec918cef.jpg
img\중독연구소/thumbnail_202x120_e1df83d4-b26f-4584-baba-6f9fec918cef.jpg (17570))
38 https://shared-comic.pstatic.net/thumb/webtoon/740132/59/thumbnail_202x120_d773d064-4347-40f4-8fb3-de9c547eaa1c.jpg
img\중독연구소/thumbnail_202x120_d773d064-4347-40f4-8fb3-de9c547eaa1c.jpg (23197))
39 https://shared-comic.pstatic.net/thumb/webtoon/655749/thumbnail/thumbnail_IMAG10_706e422b-6fd5-4a77-a501-cbc1da21bf09.jpg
img\중독연구소/thumbnail_IMAG10_706e422b-6fd5-4a77-a501-cbc1da21bf09.jpg (10873))
40 https:/